# Lernspark Pipeline Playground

This is a jupyter notebook play ground that will allow you to discover differnet types fo SQL commands and basic data engineering and analysis type operations on a sample of a much larger set of data. This is a common day to day task in data engineering analysis. 

## Prereqs
You should only be looking at this notebook after running the command `lernspark-data` and `lernspark-play` this is because this notebook is dependent on some set up to create your example data set zipped up in a `tar.gz` format. **You must run lernspark-data before lernspark-play**

# Part 0: Python Imports

For this notebook depending on what other packages you may include you should keep adding `import` statements into this block. It is reocmmended to group all imports into a single cell at the top of a notebook.

In [1]:
pip install pandas pyarrow fastparquet pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import tarfile
import os
import tempfile
import shutil
import pandas as pd
import sqlite3

# Part 1: Load Data into Memory

The first step will be to extract the data which is stored in `~\Downloads\examples.tar.gz` and load it into this notebooks memory. We will extract into the system tmp folder (a common operation) and

In [10]:
# Get the path to the downloads folder
downloads_folder = os.path.expanduser("~/Downloads")

# Specify the filename of the tar.gz file
filename = "examples.tar.gz"

# Construct the full path to the tar.gz file
file_path = os.path.join(downloads_folder, filename)

# Create a temporary directory
temp_dir = tempfile.mkdtemp()


try:
    # Open the tar.gz file
    with tarfile.open(file_path, "r:gz") as tar:
        # Extract all files to the temporary directory
        tar.extractall(path=temp_dir)
    print("Extraction completed.")
    print(f"Extracted files are located in: {temp_dir}")
    
    # Get the list of files in the temporary directory
    extracted_files = os.listdir(temp_dir)
    
    # Print the names of the extracted files
    print("Extracted files:")
    for file_name in extracted_files:
        print(f"\t{file_name}")
except FileNotFoundError:
    print(f"I can't find {file_path}, you need to run `lernspark-play` to create you data sample zip file")
except tarfile.ReadError:
    print(f"Error reading the tar.gz file: {file_path}")

Extraction completed.
Extracted files are located in: /var/folders/b4/f3v6ww_s0_zcm_srr9ndn7jr0000gn/T/tmpcyc0ud4e
Extracted files:
	Bowling_teams.parquet
	Football_teams.parquet
	Baseball_teams.parquet


## Load Parquet file into Python Memory
Now that we have unzipped the example data we can read it into python using the parquet modules.

In [12]:
# Pick a file name you want expect printed from above
# :: EDIT THIS LINE 
data_file = "Football_teams.parquet"
table_name = 'Football_teams'

# Specify the path to the extracted Parquet file
parquet_file = os.path.join(temp_dir, data_file)

try:
    # Read the Parquet file into a DataFrame
    df = pd.read_parquet(parquet_file)
    
    # Print the first few rows of the DataFrame
    print("First few rows of the data:")
    print(df.head())
    
    # Print the summary statistics of the DataFrame
    print("\nSummary statistics:")
    print(df.describe())
    
    # Explore the data further as needed
    # ...
except FileNotFoundError:
    print(f"Parquet file not found: {parquet_file}")
except Exception as e:
    print(f"Error loading Parquet file: {str(e)}")

First few rows of the data:
   ID                Name  Goals  Rank        Born
0   0     Edmond Schinner     39    79  1970-01-20
1  84       Twila DuBuque    100    23  1970-03-17
2  38  Vida Runolfsdottir     93    53  1970-01-15
3  45        Nathan Hyatt     65    45  1970-04-07
4  65      Green Johnston     79    62  1970-03-26

Summary statistics:
                 ID         Goals          Rank
count  37706.000000  37706.000000  37706.000000
mean      50.301225     49.992229     49.980109
std       29.104677     29.161905     29.113241
min        0.000000      0.000000      0.000000
25%       25.000000     25.000000     25.000000
50%       51.000000     50.000000     50.000000
75%       75.000000     75.000000     75.000000
max      100.000000    100.000000    100.000000


# Part 2: Explore Data with `pandas`
Now in this notebook you can craft any sort of data map-reduce operations you'd like on the data. Try building some operations 

# Part 3: Explore Data with `pyspark`

There are other data analysis tool instead of pandas and some are better suited for big data such as pyspark which is a python interface for Apache Spark.

# Part 4: Refactor Work into SQL Queries
Now that you have explored the data multiple different ways you can solidify the map-reduce operations into a series of SQL queries that we can deploy in a pipeline.

In [13]:
# Connect to an in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Load the DataFrame into the SQLite database
df.to_sql(table_name, conn, index=False, if_exists='replace')

# Execute SQL queries
query = f"SELECT * FROM {table_name} LIMIT 10;"
result = pd.read_sql_query(query, conn)

# Display the result
print(result)

    ID                Name  Goals  Rank        Born
0    0     Edmond Schinner     39    79  1970-01-20
1   84       Twila DuBuque    100    23  1970-03-17
2   38  Vida Runolfsdottir     93    53  1970-01-15
3   45        Nathan Hyatt     65    45  1970-04-07
4   65      Green Johnston     79    62  1970-03-26
5   14     Kattie Luettgen     13    20  1970-01-20
6  100        Brenda Davis     23    16  1970-04-07
7   39    Valentine Renner     83    11  1970-02-20
8   47      Stephen Harber     19    92  1970-02-01
9   59    Guiseppe Douglas     12    79  1970-03-31


# Part N: Clean-up
After we are done with out analysis we need to clean up our disk memory that we have created. While computers have lots and lots of memory today. This practice is good to keep as you never know what your application will be run on!

In [6]:
# Clean up the extracted files
shutil.rmtree(temp_dir)
print(f"Temporary directory {temp_dir} cleaned up.")

Temporary directory /var/folders/b4/f3v6ww_s0_zcm_srr9ndn7jr0000gn/T/tmp33f5n0es cleaned up.
